In [1]:
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle

# Here we explore the traffic data for Basel. The file is rather large (~500 MiB)

In [2]:
import pandas as pd

In [3]:
df_verkehr= pd.read_csv('../data/raw/verkehrsdaten_basel.csv', encoding = "ISO-8859-1", delimiter=";")

#### What columns exists here?

In [4]:
df_verkehr.columns

Index(['SiteCode', 'SiteName', 'DirectionName', 'LaneCode', 'LaneName', 'Date',
       'TimeFrom', 'TimeTo', 'ValuesApproved', 'ValuesEdited', 'TrafficType',
       'Total', 'MR', 'PW', 'PW+', 'Lief', 'Lief+', 'Lief+Aufl.', 'LW', 'LW+',
       'Sattelzug', 'Bus', 'andere'],
      dtype='object')

#### We will only use time, date and total, all others are split up between types of traffic. May be interesting for further use.

#### In this file, date and time are in seperate values. We need it combined into a variable so we marry the strings together. There are two time values, I arbitrarily choose TimeTo instead of TimeFrom

In [5]:
df_verkehr.Date+" "+df_verkehr.TimeTo

0          21.08.2014 01:00
1          21.08.2014 02:00
2          21.08.2014 03:00
3          21.08.2014 04:00
4          21.08.2014 05:00
5          21.08.2014 06:00
6          21.08.2014 07:00
7          21.08.2014 08:00
8          21.08.2014 09:00
9          21.08.2014 10:00
10         21.08.2014 11:00
11         21.08.2014 12:00
12         21.08.2014 13:00
13         21.08.2014 14:00
14         21.08.2014 15:00
15         21.08.2014 16:00
16         21.08.2014 17:00
17         21.08.2014 18:00
18         21.08.2014 19:00
19         21.08.2014 20:00
20         21.08.2014 21:00
21         21.08.2014 22:00
22         21.08.2014 23:00
23         21.08.2014 00:00
24         22.08.2014 01:00
25         22.08.2014 02:00
26         22.08.2014 03:00
27         22.08.2014 04:00
28         22.08.2014 05:00
29         22.08.2014 06:00
                 ...       
3212730    03.05.2020 19:00
3212731    03.05.2020 20:00
3212732    03.05.2020 21:00
3212733    03.05.2020 22:00
3212734    03.05.202

#### Convert to pandas timestamp. This takes a while

In [6]:
df_verkehr['timestamp']= pd.to_datetime(df_verkehr.Date+" "+df_verkehr.TimeTo, dayfirst=True) 

In [7]:
df_verkehr.timestamp

0         2014-08-21 01:00:00
1         2014-08-21 02:00:00
2         2014-08-21 03:00:00
3         2014-08-21 04:00:00
4         2014-08-21 05:00:00
5         2014-08-21 06:00:00
6         2014-08-21 07:00:00
7         2014-08-21 08:00:00
8         2014-08-21 09:00:00
9         2014-08-21 10:00:00
10        2014-08-21 11:00:00
11        2014-08-21 12:00:00
12        2014-08-21 13:00:00
13        2014-08-21 14:00:00
14        2014-08-21 15:00:00
15        2014-08-21 16:00:00
16        2014-08-21 17:00:00
17        2014-08-21 18:00:00
18        2014-08-21 19:00:00
19        2014-08-21 20:00:00
20        2014-08-21 21:00:00
21        2014-08-21 22:00:00
22        2014-08-21 23:00:00
23        2014-08-21 00:00:00
24        2014-08-22 01:00:00
25        2014-08-22 02:00:00
26        2014-08-22 03:00:00
27        2014-08-22 04:00:00
28        2014-08-22 05:00:00
29        2014-08-22 06:00:00
                  ...        
3212730   2020-05-03 19:00:00
3212731   2020-05-03 20:00:00
3212732   

#### File contains data for many traffic count points in the city, let's have a look at them

In [8]:
df_verkehr.SiteName.unique()

array(['235 A3-A35, Grenze CH-F',
       '350 Dreirosenbrücke lokal (mit Ein- Ausfahrten)',
       '352 Johanniterbrücke', '354 Wettsteinbrücke',
       '401 Luzernerring-Brücke', '402 Hochbergerstrasse 55',
       '403 Heuwaage-Viadukt', '404 Viaduktstrasse 60 (Rialto)',
       '405 Dorenbachviadukt', '406 Wasgenring 62',
       '407 Äussere Baselstrasse 381 (Riehen)',
       '408 Bäumlihofstrasse (Riehen)', '409 Grenzacherstrasse 511',
       '410 St. Alban-Anlage 72', '411 Fasanenstrasse (DB-Brücke)',
       '412 Dornacherstrasse 49', '413 Gundeldingerstrasse 107',
       '414 Gundeldingerstrasse 428', '415 Grosspeterstrasse 45',
       '416 J. Burckhardt-Strasse 85',
       '416 J. Burckhardt-Strasse 85 (2-spurig bis Mai 2016)',
       '417 Sevogelstrasse 82', '419 Riehenring 120',
       '420 Nauenstrasse 73', '651 Entenweidstrasse 74',
       '653 Flughafenstrasse 30', '656 Elsässerstrasse 146',
       '659 Schlachthofstrasse', '660 Flughafenstrasse, Grenze CH-F',
       '672 Zol

#### Let's create a new, smaller dataframe with only data from one location which is near to the air quality measurement stations

In [9]:
df_verkehr_stjohann = df_verkehr[df_verkehr.SiteName == "660 Flughafenstrasse, Grenze CH-F"]

#### How many data points are inside?

In [10]:
df_verkehr_stjohann.shape

(97632, 24)

#### There are approx twice as many datapoints because data is measured in two traffic lanes

In [11]:
df_grouped_mean = df_verkehr_stjohann.groupby(["TimeTo", "LaneName"]).Total.mean().reset_index()

In [12]:
df_grouped_mean.head()

,TimeTo,LaneName,Total
0,00:00,Spur 1,211.587021
1,00:00,Spur 2,60.489184
2,01:00,Spur 1,54.068338
3,01:00,Spur 2,12.903147
4,02:00,Spur 1,7.809735


In [31]:
px.line(df_grouped_mean, x=df_grouped_mean.TimeTo, y="Total", color="LaneName", color_discrete_sequence=["darkgreen", "yellow"])

#### One can clearly see the difference in lanes. In morning people go to the city (Spur 2), in the evening back (Spur 1)

#### Let's have a look at the difference of the traffic depening on the location within the city

In [14]:
df_v1 = df_verkehr[df_verkehr.LaneName == "Spur 2"].groupby(["SiteName", "TimeTo"]).Total.mean().reset_index()

In [15]:
df_v1.head()

,SiteName,TimeTo,Total
0,"235 A3-A35, Grenze CH-F",00:00,282.493282
1,"235 A3-A35, Grenze CH-F",01:00,163.632438
2,"235 A3-A35, Grenze CH-F",02:00,94.236084
3,"235 A3-A35, Grenze CH-F",03:00,68.745681
4,"235 A3-A35, Grenze CH-F",04:00,69.779750


In [16]:
px.line(df_v1, x="TimeTo", y="Total", color="SiteName")

#### And now look at the change of the mean traffic during the weekdays for a particular location and lane

In [17]:
df_v2 = df_verkehr_stjohann[df_verkehr_stjohann.LaneName == "Spur 2"].set_index("timestamp")

In [18]:
import calendar

In [29]:
px.line(x=list(calendar.day_abbr), y=df_v2.groupby(df_v2.index.weekday).Total.mean(), color_discrete_sequence=["darkgreen"])

In [20]:
list(calendar.day_abbr)

['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [21]:
df_verkehr.to_pickle("../data/clean/df_verkehr.pkl")

In [22]:
df_verkehr_stjohann.to_pickle("../data/clean/df_verkehr_stjohann.pkl")

In [23]:
df_verkehr_stjohann_index = df_verkehr_stjohann.set_index("timestamp")

In [24]:
df_christmas = df_verkehr_stjohann_index[df_verkehr_stjohann_index.LaneName == "Spur 1"].loc['2014-12-23 01:00:00' : '2014-12-27 23:00:00'].Total.reset_index()

In [25]:
df_christmas.shape

(119, 2)

In [36]:
px.scatter(df_christmas.sort_index(), x="timestamp", y="Total", color_discrete_sequence=["darkred"])